In [1]:
pip install opencv-python mediapipe numpy scikit-learn Pillow PyQt5

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import deque
import time

# PyQt5 관련 모듈
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QHBoxLayout, QVBoxLayout, QTextEdit
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QThread, pyqtSignal, Qt

# 키보드 제어 라이브러리
import pyautogui
import pyperclip

# ======================= PART 01. 수어 인식 모델 및 데이터 처리 로직 =======================

# >> 상수 및 설정 <<
consonant_labels = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
vowel_labels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ','ㅣ','ㅚ','ㅟ','ㅢ']
command_labels = ['shift', 'space', 'end']
labels = consonant_labels + vowel_labels + command_labels

font_path = "C:/Windows/Fonts/gulim.ttc"
dataset_file = 'member_hands.csv'

first_spelling = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
second_spelling = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
last_spelling = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

start_kr = 44032
end_kr = 55203

def is_hangul(char):
    return start_kr <= ord(char) <= end_kr

def decompose(char):
    if not is_hangul(char): return None, None, None
    code = ord(char) - start_kr
    last_index = code % 28
    code //= 28
    second_index = code % 21
    first_index = code // 21
    return first_spelling[first_index], second_spelling[second_index], last_spelling[last_index]

def compose(first, second, last=' '):
    try:
        first_index = first_spelling.index(first)
        second_index = second_spelling.index(second)
        last_index = last_spelling.index(last)
        code = start_kr + (first_index * 588) + (second_index * 28) + last_index
        return chr(code)
    except (ValueError, IndexError):
        return None



class HangulAssembler:
    def __init__(self):
        self.full_text = ""
        self.command_shift = False

        # 각 조합 규칙에 대한 매핑 테이블
        self.double_consonant_map = {'ㄱ': 'ㄲ', 'ㄷ': 'ㄸ', 'ㅂ': 'ㅃ', 'ㅅ': 'ㅆ', 'ㅈ': 'ㅉ'}
        self.complex_last_map = {('ㄱ', 'ㅅ'): 'ㄳ', ('ㄴ', 'ㅈ'): 'ㄵ', ('ㄴ', 'ㅎ'): 'ㄶ', ('ㄹ', 'ㄱ'): 'ㄺ', ('ㄹ', 'ㅁ'): 'ㄻ', ('ㄹ', 'ㅂ'): 'ㄼ', ('ㄹ', 'ㅅ'): 'ㄽ', ('ㄹ', 'ㅌ'): 'ㄾ', ('ㄹ', 'ㅍ'): 'ㄿ', ('ㄹ', 'ㅎ'): 'ㅀ', ('ㅂ', 'ㅅ'): 'ㅄ'}
        self.dipthong_map = {('ㅗ', 'ㅏ'): 'ㅘ', ('ㅗ', 'ㅐ'): 'ㅙ', ('ㅗ', 'ㅣ'): 'ㅚ', ('ㅜ', 'ㅓ'): 'ㅝ', ('ㅜ', 'ㅔ'): 'ㅞ', ('ㅜ', 'ㅣ'): 'ㅟ', ('ㅡ', 'ㅣ'): 'ㅢ'}
        
        # 연음 법칙을 위한 겹받침 분해 맵
        self.last_decompose_map = {v: k for k, v in self.complex_last_map.items()}

    def add_char(self, char):
        """인식된 글자(자음/모음/명령어)를 받아 전체 텍스트를 업데이트."""
        if char in command_labels:
            self._process_command(char)
        elif char in consonant_labels or char in self.double_consonant_map.values(): # 쌍자음도 처리
            self._process_consonant(char)
        elif char in vowel_labels:
            self._process_vowel(char)
        
        return self.full_text

    def _process_command(self, char):
        if char == 'shift':
            self.command_shift = True
        elif char == 'space':
            self.full_text += " "
        elif char == 'end':
            self.full_text += ".\n"

    def _process_consonant(self, char):
        # 1. 쌍자음 처리
        if self.command_shift and char in self.double_consonant_map:
            char = self.double_consonant_map[char]
        self.command_shift = False

        last_char = self.full_text[-1] if self.full_text else None
        
        # 2. 받침(종성) 추가 또는 겹받침 처리
        if last_char and is_hangul(last_char):
            first, second, last = decompose(last_char)
            # 2-1. 기존에 받침이 없는 경우 -> 새 받침 추가
            if last == ' ' and char in last_spelling:
                composed_char = compose(first, second, char)
                if composed_char: self.full_text = self.full_text[:-1] + composed_char
                return
            # 2-2. 기존에 받침이 있는 경우 -> 겹받침 시도
            elif last in last_spelling and (last, char) in self.complex_last_map:
                complex_last = self.complex_last_map[(last, char)]
                composed_char = compose(first, second, complex_last)
                if composed_char: self.full_text = self.full_text[:-1] + composed_char
                return
        
        # [수정 1] 겹받침 등 모든 조합에 실패했을 경우, 새 글자로 추가 
        self.full_text += char

    def _process_vowel(self, char):
        self.command_shift = False
        last_char = self.full_text[-1] if self.full_text else None

        if not last_char:
            # 모음으로 시작할 경우 'ㅇ'을 초성으로 추가
            composed_char = compose('ㅇ', char)
            if composed_char: self.full_text += composed_char
            else: self.full_text += char # 조합 실패 시 모음만 추가
            return

        # [수정 2] 1. 마지막 글자가 자음인 경우 -> 자음+모음 조합
        # last_spelling 대신 first_spelling을 사용하여 모든 쌍자음(ㄸ, ㅃ, ㅉ)을 처리
        if last_char in first_spelling:
            composed_char = compose(last_char, char)
            if composed_char: self.full_text = self.full_text[:-1] + composed_char
            return

        if is_hangul(last_char):
            first, second, last = decompose(last_char)
            # 2. 연음 법칙 처리 (받침이 있는 경우)
            if last != ' ':
                # 2-1. 겹받침인 경우 -> 분리 후 연음
                if last in self.last_decompose_map:
                    last_1, last_2 = self.last_decompose_map[last]
                    syllable_1 = compose(first, second, last_1)
                    syllable_2 = compose(last_2, char)
                    if syllable_1 and syllable_2: self.full_text = self.full_text[:-1] + syllable_1 + syllable_2
                # 2-2. 홑받침인 경우 -> 받침을 다음 글자 초성으로
                else:
                    syllable_1 = compose(first, second) # 받침 없는 글자
                    syllable_2 = compose(last, char) # 받침이 초성이 된 새 글자
                    if syllable_1 and syllable_2: self.full_text = self.full_text[:-1] + syllable_1 + syllable_2
                return
            # 3. 이중모음 처리 (받침이 없는 경우)
            else:
                if (second, char) in self.dipthong_map:
                    diphthong = self.dipthong_map[(second, char)]
                    composed_char = compose(first, diphthong)
                    if composed_char: self.full_text = self.full_text[:-1] + composed_char
                    return
        
        # 4. 위 조건에 해당 없으면 새 글자로 추가 (모음으로 시작하는 것과 동일하게 처리)
        composed_char = compose('ㅇ', char)
        if composed_char: self.full_text += composed_char
        else: self.full_text += char


# >> mediapipe Hands 모델 로드 및 초기화 <<
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# >> 특징 추출 유틸리티 함수들 <<
def putText_korean(image, text, pos, font_path, font_size, color):
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_size)
    draw.text(pos, text, font=font, fill=tuple(color[::-1]))
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def calculate_angles(joint):
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    angle = np.arccos(np.einsum('nt,nt->n',
        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    angle = np.degrees(angle)
    return angle.astype(np.float32)

def calculate_distances(joint):
    thumb_tip = joint[4]
    other_tips = joint[[8, 12, 16, 20]]
    distances = np.linalg.norm(other_tips - thumb_tip, axis=1)
    return distances.astype(np.float32)

def calculate_orientation_vectors(joint):
    v_direction = joint[9] - joint[0]
    if np.linalg.norm(v_direction) == 0: v_direction = np.zeros(3)
    else: v_direction = v_direction / np.linalg.norm(v_direction)

    v1 = joint[5] - joint[0]
    v2 = joint[17] - joint[0]
    v_normal = np.cross(v1, v2)
    if np.linalg.norm(v_normal) == 0: v_normal = np.zeros(3)
    else: v_normal = v_normal / np.linalg.norm(v_normal)

    return np.concatenate([v_direction, v_normal]).astype(np.float32)

# >> 데이터 로드 및 전처리 <<
def load_and_preprocess(dataset_file):
    print("데이터셋 로드 및 전처리 시작...")
    try:
        labels_str = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols=0, encoding="EUC-KR", dtype=str)
        landmarks_data = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols=range(1, 127), encoding="EUC-KR").astype(np.float32)
    except Exception as e:
        print(f"데이터 파일 로드 오류: {e}")
        return None, None, None

    all_features = []
    for row in landmarks_data:
        lh_landmarks = row[:63].reshape(21, 3); rh_landmarks = row[63:].reshape(21, 3)

        lh_angles = calculate_angles(lh_landmarks) if np.any(lh_landmarks) else np.zeros(15, dtype=np.float32)
        rh_angles = calculate_angles(rh_landmarks) if np.any(rh_landmarks) else np.zeros(15, dtype=np.float32)
        lh_coords = (lh_landmarks[1:] - lh_landmarks[0]).flatten() if np.any(lh_landmarks) else np.zeros(60, dtype=np.float32)
        rh_coords = (rh_landmarks[1:] - rh_landmarks[0]).flatten() if np.any(rh_landmarks) else np.zeros(60, dtype=np.float32)
        lh_distances = calculate_distances(lh_landmarks) if np.any(lh_landmarks) else np.zeros(4, dtype=np.float32)
        rh_distances = calculate_distances(rh_landmarks) if np.any(rh_landmarks) else np.zeros(4, dtype=np.float32)
        lh_orientation = calculate_orientation_vectors(lh_landmarks) if np.any(lh_landmarks) else np.zeros(6, dtype=np.float32)
        rh_orientation = calculate_orientation_vectors(rh_landmarks) if np.any(rh_landmarks) else np.zeros(6, dtype=np.float32)

        features = np.concatenate([lh_angles, rh_angles, lh_coords, rh_coords, lh_distances, rh_distances, lh_orientation, rh_orientation])
        all_features.append(features)

    all_features = np.array(all_features, dtype=np.float32)
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels_str)

    print("데이터 전처리 완료!")
    return all_features, encoded_labels, encoder

# >> 모델 학습 <<
def train_model(dataset_file):
    X, y, encoder = load_and_preprocess(dataset_file)
    if X is None: return None, None

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    print("--- 랜덤 포레스트 모델 학습 시작 ---")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print("\n--- 학습 완료 ---")
    print(f"데이터 특징 차원: {X.shape[1]}")
    print(f"모델 테스트 정확도: {accuracy * 100:.2f}%")
    return model, encoder

# ======================= PART 02. PyQt5 GUI 및 영상 처리 스레드 =======================

class VideoThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)
    update_text_signal = pyqtSignal(str)

    def __init__(self, model, encoder):
        super().__init__()
        self._run_flag = True
        self.model = model
        self.encoder = encoder
        
        # [수정 1] 새로운 변수 추가
        self.last_recognition_time = 0  # 마지막으로 제스처를 인식한 시간
        self.RECOGNITION_COOLDOWN = 1.5 # 인식 후 쿨타임 (1.5초). 이 값을 조절하여 인식 속도 변경 가능

    def run(self):
        history = deque(maxlen=5)
        display_label = ""
        display_start_time = None
        display_duration = 1.5 # 쿨타임과 비슷하게 조정

        cap = cv2.VideoCapture(0)
        with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
            while self._run_flag and cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    continue

                frame = cv2.flip(frame, 1)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                result = hands.process(frame_rgb)
                guide_text = "손을 보여주세요"
                current_time = time.time() # 현재 시간 측정

                if result.multi_hand_landmarks:
                    # ... (특징 추출 부분은 기존과 동일) ...
                    init_zeros = {'angles': np.zeros(15), 'coords': np.zeros(60), 'distances': np.zeros(4), 'orientation': np.zeros(6)}
                    lh_features, rh_features = init_zeros.copy(), init_zeros.copy()

                    for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
                        handedness = result.multi_handedness[i].classification[0].label
                        joint = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
                        features = {
                            'angles': calculate_angles(joint), 'coords': (joint[1:] - joint[0]).flatten(),
                            'distances': calculate_distances(joint), 'orientation': calculate_orientation_vectors(joint)
                        }
                        if handedness == "Left": lh_features = features
                        elif handedness == "Right": rh_features = features
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    feature_vector = np.concatenate([
                        lh_features['angles'], rh_features['angles'], lh_features['coords'], rh_features['coords'],
                        lh_features['distances'], rh_features['distances'], lh_features['orientation'], rh_features['orientation']
                    ]).reshape(1, -1)

                    predicted_index = self.model.predict(feature_vector)
                    predicted_label = self.encoder.inverse_transform(predicted_index)[0]
                    history.append(predicted_label)

                    # [수정 2] 제스처 인식 및 전달 로직 변경
                    if len(history) == 5 and len(set(history)) == 1:
                        # 쿨타임이 지났는지 확인
                        if current_time - self.last_recognition_time > self.RECOGNITION_COOLDOWN:
                            recognized_gesture = history[-1]
                            print(f"Recognized: {recognized_gesture} at {current_time:.2f}")

                            # GUI에 신호 전달
                            self.update_text_signal.emit(recognized_gesture)

                            # 화면 표시용 변수 업데이트 및 쿨타임 타이머 초기화
                            display_label = recognized_gesture
                            display_start_time = current_time
                            self.last_recognition_time = current_time # 마지막 인식 시간 업데이트
                            history.clear()

                if display_start_time and ((current_time - display_start_time) < display_duration):
                    display_text = display_label
                else:
                    display_text = guide_text

                frame = putText_korean(frame, display_text, (50, 420), font_path, 40, (0, 255, 0))
                self.change_pixmap_signal.emit(frame)

        cap.release()

    def stop(self):
        self._run_flag = False
        self.wait()

class HandGestureApp(QWidget):
    def __init__(self, model, encoder):
        super().__init__()
        self.setWindowTitle("수어 번역 프로그램 ('q'를 눌러 종료)")
        self.display_width = 640
        self.display_height = 480

        # UI 요소 생성
        self.image_label = QLabel(self)
        self.image_label.resize(self.display_width, self.display_height)
        self.image_label.setStyleSheet("border: 2px solid black;")

        self.chat_box = QTextEdit(self)
        self.chat_box.setReadOnly(True)
        self.chat_box.setFixedWidth(200) # ⭐ 대화창 너비 고정
        self.chat_box.setStyleSheet("font-size: 20px; border: 2px solid black;")

        # ⭐ 수평 레이아웃으로 변경
        hbox = QHBoxLayout()
        hbox.addWidget(self.image_label)
        hbox.addWidget(self.chat_box)
        self.setLayout(hbox)

        # HangulAssembler 인스턴스 생성
        self.assembler = HangulAssembler()

        # 비디오 스레드 생성 및 시작
        self.thread = VideoThread(model, encoder)
        self.thread.change_pixmap_signal.connect(self.update_image)
        self.thread.update_text_signal.connect(self.update_chat)
        self.thread.start()

    def update_image(self, cv_img):
        qt_img = self.convert_cv_qt(cv_img)
        self.image_label.setPixmap(qt_img)

    def update_chat(self, new_char):
        # assembler에 새 글자 추가하고, 반환된 전체 텍스트로 화면을 업데이트
        full_text = self.assembler.add_char(new_char)
        self.chat_box.setText(full_text)
        self.chat_box.verticalScrollBar().setValue(self.chat_box.verticalScrollBar().maximum()) # 자동 스크롤

        

    def convert_cv_qt(self, cv_img):
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        convert_to_Qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        p = convert_to_Qt_format.scaled(self.display_width, self.display_height, Qt.KeepAspectRatio)
        return QPixmap.fromImage(p)

    # 'q' 키를 누르면 종료 (이벤트 핸들러)
    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Q:
            self.close()

    def closeEvent(self, event):
        self.thread.stop()
        event.accept()

# ======================= PART 03. 메인 실행 부분 =======================

if __name__ == "__main__":
    # 1. 모델 학습
    trained_model, label_encoder = train_model(dataset_file)
    
    # 2. 모델 학습 성공 시 GUI 앱 실행
    if trained_model and label_encoder:
        app = QApplication(sys.argv)
        window = HandGestureApp(trained_model, label_encoder)
        window.show()
        sys.exit(app.exec_())
    else:
        print("모델 학습에 실패하여 프로그램을 종료합니다.")